In [1]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging
from IPython.display import Markdown, display

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration
from deriva_ml import MLVocab as vc
from deriva_ml.deriva_definitions import ColumnDefinition, BuiltinTypes
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2025-01-19 11:30:06.117569: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-19 11:30:06.117617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-19 11:30:06.118501: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-19 11:30:06.125015: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-19 11:30:06.880563: W tensorflow/compiler/tf2

In [2]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'www.eye-ai.org'
# host = 'dev.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2025-01-19 11:30:07,431 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-01-19 11:30:07,432 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


In [3]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-01-19 11:30:10,592 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-01-19 11:30:10,593 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


# Configuration

In [4]:
# Workflow instance
cur_workflow = Workflow(
    name="Create slected image datasets",
    url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/multimodal/selected_image_datasets.ipynb",
    workflow_type="multimodal_feature_engineering"
)
# Configuration instance.

# Set to False if you only need the metadata from the bag, and not the assets.
download_assets = True

config = ExecutionConfiguration(
    # Comment out the following line if you don't need the assets.
    # datasets=[source_dataset] if download_assets else [],
    assets=['4-3S3G'], # dev: 5-SJFY prod: 4-3S3G
    workflow=cur_workflow,
    description="Create slected image datasets on dev")

# Initialize execution
execution = EA.create_execution(config)



2025-01-19 11:30:13,893 - INFO - Configuration validation successful!
2025-01-19 11:30:15,123 - INFO - File [/data/lizhiwei/EyeAI_working/4-410P/asset/feature_angle_laterality.csv] transfer successful. 54.02 KB transferred. Elapsed time: 0:00:00.001715.
2025-01-19 11:30:15,124 - INFO - Verifying MD5 checksum for downloaded file [/data/lizhiwei/EyeAI_working/4-410P/asset/feature_angle_laterality.csv]


In [5]:
print(execution)

caching_dir: /data
working_dir: /data/lizhiwei/EyeAI_working
execution_rid: 4-410P
workflow_rid: 4-410M
dataset_paths: []
asset_paths: [PosixPath('/data/lizhiwei/EyeAI_working/4-410P/asset/feature_angle_laterality.csv')]
configuration: datasets=[] assets=['4-3S3G'] workflow=Workflow(name='Create slected image datasets', url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/multimodal/selected_image_datasets.ipynb', workflow_type='multimodal_feature_engineering', version=None, description=None) description='Create slected image datasets on dev'


In [6]:
feature = pd.read_csv(execution.asset_paths[0])
feature.dropna(subset=['Image'], inplace=True)
feature.drop_duplicates(subset=['Image'], inplace=True)
selected_image = feature['Image'].tolist()

In [7]:

def retrieve_image_rids(dataset_rid):
    eye_ai = EA.domain_path
    dataset_sub = eye_ai.Subject_Dataset
    sub = eye_ai.Subject
    obs = eye_ai.Observation
    image = eye_ai.Image
    
    result = dataset_sub.path.filter(dataset_sub.Dataset == dataset_rid).\
    link(sub, on=dataset_sub.Subject==sub.RID).\
    link(obs, on=sub.RID==obs.Subject).\
    link(image, on=obs.RID==image.Observation)
    return pd.DataFrame(result.entities())['RID'].to_list()

train = '2-C9PP'
test = '2-C9PR'
train_list = retrieve_image_rids(train)
test_list = retrieve_image_rids(test)
train_selected_list = [rid for rid in selected_image if rid in train_list]
test_selected_list = [rid for rid in selected_image if rid in test_list]

In [12]:
train_image = execution.create_dataset(['USC', 'Image', 'Training'], 
                                       description='Selected images for multimodal training')
test_image = execution.create_dataset(['USC', 'Image', 'Test'], 
                                      description='Selected images for multimodal test')

In [14]:
EA.add_dataset_members(dataset_rid=train_image, members=train_selected_list)
EA.add_dataset_members(dataset_rid=test_image, members=test_selected_list)

'4-411G'

In [15]:
EA.add_dataset_members(dataset_rid=train, members= [train_image])
EA.add_dataset_members(dataset_rid=test, members= [test_image])

'2-C9PR'

# Upload results

In [16]:
# save assets to asset_path

# upload assets to catalog
execution.upload_execution_outputs(clean_folder=True)

2025-01-19 11:38:30,469 - INFO - Initializing uploader: GenericUploader v1.7.5 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2025-01-19 11:38:30,470 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-01-19 11:38:30,471 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-01-19 11:38:30,482 - INFO - Scanning files in directory [/data/lizhiwei/EyeAI_working/deriva-ml/execution/4-410P/execution-asset]...
2025-01-19 11:38:30,483 - INFO - File upload processing completed: 0 files were uploaded successfully, 0 files failed to upload due to errors, 0 files were skipped because they did not satisfy the matching criteria of the configuration.
2025-01-19 11:38:30,554 - INFO - Initializing uploader: GenericUploader v1.7.5 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x

{'Execution_Config/configuration.json': FileUploadState(state=<UploadState.success: 0>, status='Complete', result={'URL': '/hatrac/execution_metadata/581715425ede233abbce9e37caa6b029.configuration.json:Kzldu6WIffBPTyrGeTuYEWEFudhDuuc0', 'RID': '4-44EP', 'RCT': '2025-01-19T19:38:31.124398+00:00', 'RMT': '2025-01-19T19:38:31.124398+00:00', 'RCB': 'https://auth.globus.org/3769492a-b197-4063-952e-2500351dbcdd', 'RMB': 'https://auth.globus.org/3769492a-b197-4063-952e-2500351dbcdd', 'Filename': 'configuration.json', 'Description': None, 'Length': 373, 'MD5': '581715425ede233abbce9e37caa6b029', 'Execution_Metadata_Type': 'Execution_Config'}, rid='4-44EP'),
 'Runtime_Env/environment_snapshot_pl_clz_c.txt': FileUploadState(state=<UploadState.success: 0>, status='Complete', result={'URL': '/hatrac/execution_metadata/ff510496e7048767ace690657ae64bce.environment_snapshot_pl_clz_c.txt:cH2z3rBVa6M8OqLGFeqGFP9QyJm1ZYs0', 'RID': '4-44ER', 'RCT': '2025-01-19T19:38:31.355351+00:00', 'RMT': '2025-01-19T1

In [13]:
# eye_ai = EA.domain_path
eye_ai = EA.pathBuilder.schemas['eye-ai']
ml = EA.pathBuilder.schemas['deriva-ml']
dataset = ml.Dataset

dataset_sub = eye_ai.Subject_Dataset
sub = eye_ai.Subject
obs = eye_ai.Observation
image = eye_ai.Image

result = dataset.path.filter(dataset.RID == '2-7P5P').\
result
link(dataset_sub, on=dataset.RID==dataset_sub.Dataset).\
link(sub, on=dataset_sub.Subject==sub.RID).\
link(obs, on=sub.RID==obs.Subject).\
link(image, on=obs.RID==image.Observation)
catalog_image = pd.DataFrame(result.entities())[['RID', 'MD5']]

ValueError: 'right' is from a different catalog. Cannot link across catalogs.